In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [13]:
import sys

In [10]:
# load ascii text and covert to lowercase
filename = "compression_data.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  616
Total Vocab:  29


In [3]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  516


In [4]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [5]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [6]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [7]:
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
516/516 [==============================] - 7s 13ms/step - loss: 3.3488

Epoch 00001: loss improved from inf to 3.34880, saving model to weights-improvement-01-3.3488.hdf5
Epoch 2/20
516/516 [==============================] - 3s 5ms/step - loss: 3.2053

Epoch 00002: loss improved from 3.34880 to 3.20532, saving model to weights-improvement-02-3.2053.hdf5
Epoch 3/20
516/516 [==============================] - 3s 6ms/step - loss: 3.0612

Epoch 00003: loss improved from 3.20532 to 3.06117, saving model to weights-improvement-03-3.0612.hdf5
Epoch 4/20
516/516 [==============================] - 3s 5ms/step - loss: 3.0614

Epoch 00004: loss did not improve from 3.06117
Epoch 5/20
516/516 [==============================] - 3s 5ms/step - loss: 3.0293

Epoch 00005: loss improved from 3.06117 to 3.02929, saving model to weights-improvement-05-3.0293.hdf5
Epoch 6/20
516/516 [==============================] - 3s 5ms/step - loss: 2.9643

Epoch 00006: loss improved from 3.02929 to 2.96427, 

In [14]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" onder what you are!
up above the world so high,
like a diamond in the sky.

when this blazing sun is "
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         